In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import time
from random import randint
import re


In [3]:
def gettingUrl(url):

    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    # print(userAgent)
    options.headless = True  # setting this to true makes the browser work in the background
    options.add_argument("--proxy-server='direct://'")
    options.add_argument("--proxy-bypass-list=*")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument('--lang=en_US')
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-extensions")
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # DRIVER_PATH = "/usr/bin/chromedriver"
    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=options)
    driver.get(url)
    driver.get_screenshot_as_file("screenshot1.png")
    time.sleep(randint(10,30))

    html = driver.page_source
    # this renders the JS code and stores all of this information in static HTMl code.
    return html


In [ ]:
# extracting data from acs nano site
html = gettingUrl("https://pubs.acs.org/doi/10.1021/acsnano.5b05040")


In [ ]:
# finding out the number of times the research paper number 5 is mentioned in the text.
# creating an instance of the BeautifulSoup library
soup_new = BeautifulSoup(html, 'lxml')
a_tags = soup_new.find('div', class_="article_content-left ui-resizable")
# print(a_tags)
count = a_tags.find_all('a', class_="ref5")
print(len(count))

In [ ]:
#to find the last author names of the references
soup_new = BeautifulSoup(html, 'lxml')
allLastAuthors =[]

relevant = soup_new.find('div', class_="article_content-left ui-resizable")
string_Refs = relevant.find('p', class_="references-count")
string = string_Refs.text
number_Refs = re.findall(r'\d+', string)

print(number_Refs[0])

i=0

for i in range(1, int(number_Refs[0])+1):

    info = relevant.find('li', {"id":"ref"+str(i)})

    if info is None:
        allLastAuthors.append("No information")
    # print(info)

    authors = info.find_all('span',class_='NLM_contrib-group')
    # print(authors)
    last_author = authors[-1].text
    allLastAuthors.append(last_author)
    i+=1

# print((allLastAuthors))

In [ ]:
# creating an instance of the BeautifulSoup library
soup2 = BeautifulSoup(html, 'lxml')
links = soup2.find('div', class_="article_content-left ui-resizable")
# creating a list containing the google scholar links for all the reference papers.
paperLinksgoogleScholar = links.find_all('a', class_="google-scholar")
scholarLinks = []
for link in paperLinksgoogleScholar:
    # A new list containing the google scholar links for only the first three papers is created.
    scholarLinks.append(link['href'])
for link in scholarLinks:
    index = scholarLinks.index(link)
    print(index+1, link)


In [ ]:
authors_new = []
for link in scholarLinks:
    html = gettingUrl(link)
    # creating an instance of the BeautifulSoup library
    soup3 = BeautifulSoup(html, 'lxml')
    content = soup3.find('div', class_="gs_a")

    if content == None:
        print("No Data Found")
    else:
        authors = content.find_all('a')

        # getting the name of the professor to then set up further relationships.
        print(authors[-1].text)
        authors_new.append(authors[-1].text)

In [ ]:
# finding out the publication house link of the paper from google scholar.
paperLink = soup3.find('h3', class_='gs_rt')
print(paperLink.a['href'])


In [ ]:
# finding out the name of the research paper by extracting the text from the google scholar page.

options = Options()
ua = UserAgent
userAgent = ua
options.headless = True
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument('--lang=en_US')
options.add_argument("--window-size=1920,1080")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--disable-extensions")
options.add_argument(f'user-agent={userAgent}')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)
driver.get(scholarLinks[4])
# driver.find_element(By.XPATH, value='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[2]').click()
driver.find_element(By.LINK_TEXT, "Cite").click()
time.sleep(randint(10, 30))

# To get the name of the 5th paper
html = gettingUrl(driver.current_url)
# print(driver.current_url)
soup4 = BeautifulSoup(html, 'lxml')
paperName = soup4.find('div', class_='gs_citr')
print(paperName.i.text)  # printing the name of the 5th paper.

# quits the whole browser session along with all the associated browser windows, tabs and pop-ups.
driver.quit()


In [ ]:
# Extraction from "Nature" publication house.
html = gettingUrl("https://www.nature.com/articles/s41417-022-00495-w")


# to get the google scholar links of the references.
paperLinksgoogleScholarnature = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'html.parser')
links = soup.find('ol', class_="c-article-references")
tags = links.find_all('a', {'data-track-action': 'google scholar reference'})
for tag in tags:
    # creating a list containing the google scholar links for all the
    paperLinksgoogleScholarnature.append(tag['href'])


# to get the doi of the references.
natureDOI = []
refNumber = len(paperLinksgoogleScholarnature)

i=0

for i in range(1, int(refNumber)+1):
    doiLinks = links.find('a', {'aria-label': 'Article reference '+str(i)})

    if doiLinks == None:
        natureDOI.append("No DOI")
    else:
        natureDOI.append(doiLinks['href'])
    i+=1


# extracting the authors from the main page itself.
text = soup.find_all('ol', class_='c-article-references')
# print(text)
text_main = soup.find_all(
    'li', class_='c-article-references__item js-c-reading-companion-references-item')
# print(text_main[0])

Titles = []
allLastAuthors_nature = []
journalName = []
yearPublication = []

for ref in text_main:
    string = ref.find('p', class_='c-article-references__text')

    togetdeets = string.text.split('.')

    # returns authors in the form of a string.
    allAuthors = togetdeets[0]
    allAuthorsList = allAuthors.split(',')  # converting to a list

    Titles.append(togetdeets[1])  # gets the title of the references
    
    journalName.append(togetdeets[2])# gets the journal name of the references

    if len(togetdeets) < 4:
        yearPublication.append(' ') #gets the year of publication of the references
    else:
        year = togetdeets[3].split(';')
        yearPublication.append(year[0])

    if allAuthorsList[-1] == ' et al':
        allLastAuthors_nature.append(allAuthorsList[-2])
    else:
        allLastAuthors_nature.append(allAuthorsList[-1])

print(yearPublication)

In [ ]:
# Extraction from "Springer" publication house.
html = gettingUrl("https://link.springer.com/article/10.1007/s43673-022-00064-1")
paperLinksgoogleScholarspringer = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
links = soup.find('ol', class_="c-article-references")
refNumber = len(links)
k = 0
for k in range(1, int(refNumber)+1):
    tags = links.find('a', {'aria-label': 'Google Scholar reference ' +str(k)})
    if tags==None:
        paperLinksgoogleScholarspringer.append("No Google Scholar Link")
    else:
        paperLinksgoogleScholarspringer.append(tags['href'])
    k+=1
# print(paperLinksgoogleScholarspringer)



#to get the doi of the references.
springerDOI = []
refNumber = len(links)
# print(refNumber)

i=0

for i in range(1, refNumber+1):
    doiLinks = links.find('a', {'aria-label': 'Article reference '+str(i)})
    # print(doiLinks)

    if doiLinks == None:
        springerDOI.append("No DOI")
    else:
        springerDOI.append(doiLinks['href'])
    i+=1

# print(springerDOI)


#extracting the authors from the main page itself.
text = soup.find_all('ol', class_='c-article-references')

k = 0


text_main = soup.find_all('li', class_='c-article-references__item js-c-reading-companion-references-item')

Titles = []
allLastAuthors_springer = []
journalName = []
yearPublication = []

for ref in text_main:
    string = ref.find('p', class_="c-article-references__text")

    togetdeets = string.text.split(',')
    content = togetdeets[-2].split('.')
    Titles.append(content[0]) # gets the title of the references

    if len(togetdeets)<4:
        allLastAuthors_springer.append("No Data Found")
    else:
        allLastAuthors_springer.append(togetdeets[-3])

    sum = ''
    for j in range(1, len(content)-1):
        sum += content[j]
    journalName.append(sum) # gets the journal name of the references

    year = togetdeets[-1].split('(')
    # print(year[-1])

    if year[-1][0]==" ":
        yearPublication.append("No Data Found") # gets the year of publication of the references
    else:
        year_new = year[-1].split(')')
        yearPublication.append(year_new[0]) #gets the year of publication of the references


# print(len(springerDOI))

In [ ]:
# extraction from "Elsevier(sciencedirect)" publication house.
# html = gettingUrl("https://www.sciencedirect.com/science/article/pii/S2376060522000608")
paperLinksgoogleScholarscienceDirect = []
Titles =[]
scienceDirectDOI = []
journalName = []
links = []
allLastAuthors_sciencedirect = []
yearPublication = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find_all('dd', class_="reference")

refNumber = len(tags)

i = 0

for i in range(1, int(refNumber)+1):
    tag = soup.find('dd', {'id': 'sref'+"{}".format(i)})
    Titles.append(tag.find('strong',class_="title").text)
    journalName.append(tag.find('div',class_="host").text)
    yearPublication.appendtag.find('div',class_="host").text.split(',')[-2].split('(')[-1].split(')')[0]
    links = tag.find_all('a', class_="link")
    if links[0].text == "Article":
        scienceDirectDOI.append("https://www.sciencedirect.com/"+links[0]['href'])
    else:
        scienceDirectDOI.append("No DOI")

    if links[-1].text == "Google Scholar":
        paperLinksgoogleScholarscienceDirect.append(links[-1]['href'])
    else:
        paperLinksgoogleScholarscienceDirect.append("No Google Scholar Link")
    
    content = tag.find('div',class_="contribution").text.split(',')
    if "et al" in content[-1]:
        allLastAuthors_sciencedirect.append(content[-2])
    else:
        allLastAuthors_sciencedirect.append(content[-1].replace(Titles[i-1]," "))
    i+=1

In [ ]:
# extraction from "MDPI" publication house.
paperLinksgoogleScholarMDPI = []
toGetDeets=[]
Titles=[]
mdpiDOI = []
allLastAuthors_mdpi = []
journalName = []
yearPublication = []
# html = gettingUrl("https://www.mdpi.com/2226-4310/9/11/704/htm")
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find('ol', class_="html-xx")

refNumber = len(tags)

i = 0

for i in range(1, int(refNumber)+1):
    content = tags.find('li', {'data-content': "{}".format(i)+'.'})

    forName = content.find('span', class_="html-italic")

    if forName == None:
        journalName.append("No Data Found")
    else:
        journalName.append(forName.text)

    forYear = content.find('b')
    
    if forYear == None:
        yearPublication.append("No Data Found")
    else:
        yearPublication.append(forYear.text)

    toGetDeets.append(content.text.split('[')[0])

    if "CrossRef" in content.text:
        mdpiDOI.append(content.find('a',class_='cross-ref')['href'])
    else:
        mdpiDOI.append("DOI not found")

    if "Google Scholar" in content.text:
        paperLinksgoogleScholarMDPI.append(content.find('a',class_='google-scholar')['href'])
    else:
        paperLinksgoogleScholarMDPI.append("Google Scholar Link not found")

for deet in toGetDeets:
    detail = deet.split(';')[-1]
    allLastAuthors_mdpi.append(detail.split('.')[0])
    Titles.append(detail.split('.')[:-2])




    
# links = tags.find_all('a', class_="google-scholar")
# for link in links:
#     paperLinksgoogleScholarMDPI.append(link['href'])
#     index = links.index(link)
#     print(index+1, link['href'])


In [ ]:
# extraction from "Science" publication house.
# html = gettingUrl("https://www.science.org/doi/10.1126/sciadv.abq2104")
paperLinksgoogleScholarScienceall = []
paperLinksgoogleScholarScience = []
scienceDOI = []
toGetDeets = []
allLastAuthors_science = []
journalName = []
Titles = []
yearPublication = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find('section', {'id': 'bibliography'})
forLabels = tags.find_all("div", class_='label')

refNumber = len(forLabels)

i=0

content = tags.find_all('div', class_= "citation")
for con in content:
    toGetDeets.append(con.find('div', class_='citation-content'))

for deet in toGetDeets:
    if deet.find('em') == None:
        journalName.append("No Data Found")
    else:
        journalName.append(deet.find('em').text)

for deet in toGetDeets:
    detail = deet.text.split(',')
    if ('(' in detail[-1]) == False:
        yearPublication.append("No Data Found")
    else:
        yearPublication.append(detail[-1].split('(')[-1].split(')')[0])

for deet in toGetDeets:
    Titles.append(deet.text)

for con in content:
    link = con.find('a')
    if link.text == "Crossref":
        scienceDOI.append(link['href'])
    else:
        scienceDOI.append("No DOI")

for con in content:
    link = con.find_all('a')
    if (link[-1].text == "Google Scholar"):
        paperLinksgoogleScholarScience.append(link[-1]['href'])
    else:
        paperLinksgoogleScholarScience.append("Google Scholar Link Not Found")

for i in range(0,refNumber):
    allLastAuthors_science.append("Next Column for Authors")

In [ ]:
# extraction from "IEEE" publication house.
# html = gettingUrl("https://ieeexplore.ieee.org/document/9837920/references#references")
paperLinksgoogleScholarIEEE = []
ieeeDOI = []
toGetDeets = []
allLastAuthors_ieee = []
journalName = []
Titles = []
yearPublication = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find_all('div', class_='reference-container')
for tag in tags:
    
    detail = tag.find('div',class_="col u-px-1")
    fineDetail = detail.find('div')
    
    if fineDetail.find('em') == None:
        journalName.append("No Data Found")
    else:
        journalName.append(fineDetail.find('em').text)

    Titles.append(fineDetail.text.split('"')[-1])

    if 'and' in fineDetail.text.split('"')[0]:
        allLastAuthors_ieee.append(fineDetail.text.split('"')[0].split('and')[-1])
    else:
        allLastAuthors_ieee.append(fineDetail.text.split('"')[0])

    yearPublication.append(fineDetail.text.split('"')[-1].split('.')[-2])

    doi = tag.find('a', class_="stats-reference-link-crossRef")
    
    doi_ieee = tag.find('a', class_="stats-reference-link-viewArticle")

    if doi == None and doi_ieee == None:
        ieeeDOI.append("No DOI")
    elif doi == None:
        ieeeDOI.append("https://ieeexplore.ieee.org"+doi_ieee['href'])
    else:
        ieeeDOI.append(doi['href'])

    gscholar = tag.find('a', class_="stats-reference-link-googleScholar")
    if gscholar == None:
        paperLinksgoogleScholarIEEE.append("Google Scholar Link Not Found")
    else:
        paperLinksgoogleScholarIEEE.append(gscholar['href'])


In [ ]:
# extraction from "Cambridge publishing house" publication house.
# html = gettingUrl("https://www.cambridge.org/core/journals/advances-in-archaeological-practice/article/professionalcollector-collaboration/8DB3D024A682DEC74457D6D5708B8D73")
paperlinksgoogleScholarCambridge = []
content_list = []
allLastAuthors_cambridge = []
Titles = []
yearPublication = []
journalName = []
cambridgeDOI = []
allAuthors = []

# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find('div', {'id': 'references-list'})
for tag in tags:
    content_list.append(tag)

content = content_list[3:]
refNumber = len(content)

i = 0

for i in range(2, refNumber+2):
    toGetDeets = content[i-2].find('div',
                                   {'id': 'reference-'+"{}".format(i)+'-content'})

    link = toGetDeets.find('a', class_='ref-link')
    links = toGetDeets.find_all('a', class_='ref-link')
    if link.text == "CrossRef":
        cambridgeDOI.append(link['href'])
    else:
        cambridgeDOI.append("No DOI")

    if links[-1].text == "Google Scholar":
        paperlinksgoogleScholarCambridge.append(links[-1]['href'])
    else:
        paperlinksgoogleScholarCambridge.append(
            "Google Scholar Link Not Found")

        yearPublication.append(toGetDeets.find('span', class_='year').text)

    if toGetDeets.find('span', class_='article-title') != None:
        Titles.append(toGetDeets.find('span', class_='article-title').text)
    else:
        Titles.append("No data found")

    if toGetDeets.find('span', class_='publisher-name') != None:
        journalName.append(toGetDeets.find(
            'span', class_='publisher-name').text)
    elif toGetDeets.find('span', class_='source') != None:
        journalName.append(toGetDeets.find('span', class_='source').text)
    else:
        journalName.append("No Data Found")

    # print(toGetDeets)
    author = toGetDeets.find_all('span', class_='string-name')
    allAuthors.append(author)

for auth in allAuthors:
    if auth == []:
        allLastAuthors_cambridge.append("No Data Found")
    elif len(auth) == 1:
        allLastAuthors_cambridge.append(auth[0].text)
    else:
        allLastAuthors_cambridge.append(auth[-1].text)
# Titles
# (paperlinksgoogleScholarCambridge)


In [43]:
#extraction from "ACS" publication house.
paperlinksgoogleScholarACS = []
allLastAuthors_acs = []
Titles = []
yearPublication = []
journalName = []
acsDOI = []
DOI = []
authors = []
# html = gettingUrl("https://pubs.acs.org/doi/10.1021/acsnano.5b05040")
soup = BeautifulSoup(html, 'lxml')
tags = soup.find("ol",class_="useLabel")

refNumber = len(tags)

i = 0

for i in range(1, int(refNumber)+1):
    toGetDeets = tags.find('li', {'id': 'ref'+'{}'.format(i)})
    content = toGetDeets.find('div', {'id': 'cit'+'{}'.format(i)})
    if content == None:
        allLastAuthors_acs.append("No Data Found")
        Titles.append("No Data Found")
        yearPublication.append("No Data Found")
        journalName.append("No Data Found")
        DOI.append("No DOI")
        authors.append("No Data Found")
        paperlinksgoogleScholarACS.append("Google Scholar Link Not Found")
    else:
        authors.append(content.find_all('span', class_='NLM_contrib-group'))
        allLastAuthors_acs.append(authors[i-1][-1].text)
        Titles.append(content.find('span', class_='NLM_article-title').text)
        journalName.append(content.find('span', class_='citation_source-journal').text)
        yearPublication.append(content.find('span', class_='NLM_year').text)
        DOI.append(content.find('span',class_='refDoi').text)
    
        paperlinksgoogleScholarACS.append(content.find('a',class_='google-scholar')['href'])

j=0
for link in DOI:
    if link == "No DOI":
        acsDOI.append("No DOI")
    else:
        x= re.findall("(?<=DOI\: )(.*)", DOI[j])
        new = "https://doi.org/"
        var = ' '.join(x)
        acsDOI.append("".join([new,var]))
    j+=1

['http://scholar.google.com/scholar_lookup?hl=en&volume=306&publication_year=2004&pages=666-669&journal=%00null%00&issue=%00null%00&issn=%00null%00&author=K.+S.+Novoselovauthor=A.+K.+Geimauthor=S.+V.+Morozovauthor=D.+Jiangauthor=Y.+Zhangauthor=S.+V.+Dubonosauthor=I.+V.+Grigorievaauthor=A.+A.+Firsov&title=Electric+Field+Effect+in+Atomically+Thin+Carbon+Films&pmid=%00empty%00&doi=10.1126%2Fscience.1102896',
 'http://scholar.google.com/scholar_lookup?hl=en&volume=438&publication_year=2005&pages=201-204&journal=%00null%00&issue=%00null%00&issn=%00null%00&author=Y.+Zhangauthor=Y.-W.+Tanauthor=H.+L.+Stormerauthor=P.+Kim&title=Experimental+Observation+of+the+Quantum+Hall+Effect+and+Berry%E2%80%99s+Phase+in+Graphene&pmid=%00empty%00&doi=10.1038%2Fnature04235',
 'http://scholar.google.com/scholar_lookup?hl=en&volume=8&publication_year=2008&pages=3498-3502&journal=%00null%00&issue=%00null%00&issn=%00null%00&author=M.+D.+Stollerauthor=S.+Parkauthor=Y.+Zhuauthor=J.+Anauthor=R.+S.+Ruoff&title=Graph

In [ ]:
def Publication(link):
    x = re.findall('\.(.*?)\.', link)
    return x

print(Publication("https://link.springer.com/article/10.1007/s43673-022-00064-1"))
print(Publication("https://www.nature.com/articles/s41417-022-00495-w"))

In [ ]:
string = "https://link.springer.com/article/10.1007/s43673-022-00064-1"
string = "https://www.science.org/doi/10.1126/science.1114655"
string = "https://www.sciencedirect.com/science/article/pii/S2376060522000608"
string_main = string.split('.')

if "sciencedirect" in string_main:
    print("yes")

In [ ]:
i = 1
for i in range(1,5):
    print("{}".format(i)+'.')